## classification - 앙상블

In [ ]:
    #1) import
    import numpy as np  # 선형대수, 행렬, 벡터
    import pandas as pd  # CSV파일 읽기, DataFrame 객체, 평균, 중앙값, 분산, 표준편차, 사분위수, 상관관계
    import matplotlib.pyplot as plt  # 박스플랏, 산점도import statsmodels.formula.api as smf
    from statsmodels.graphics.mosaicplot 
    import mosaicfrom sklearn.model_selection import train_test_splitfrom sklearn 
    import treeimport pydotplusfrom IPython.display import Imagefrom sklearn.tree 
    import export_graphvizfrom sklearn.metrics import roc_auc_scorefrom sklearn.metrics 
    import roc_curvefrom sklearn.ensemble import RandomForestClassifierfrom sklearn.metrics 
    import accuracy_scorefrom sklearn.metrics import confusion_matriximport seaborn as sns
    
    #2) 데이터 가져오기
    hmeq = pd.read_csv('hmeq.txt',sep='\t')
    ##옵션 사용하여 데이터 가져오기
    ##df_pd = pd.read_csv("projectdataO.csv", header = None, encoding = 'cp949')
    
    ##데이터 탐색
    hmeq.describe()
    hmeq.head(10)
    hmeq = hmeq.drop(['REASON','JOB'], axis=1)
    
    ##변수명 변경
    ##df_pd.columns = ['X1','X2', 'X3','X4','X5', 'X6','X7','X8', 'X9','X10']
    
    ##결측치 처리
    hmeq = hmeq.fillna(hmeq.median())
    ###행제거
    ##df_sp.na.drop().show()
    ######프로젝트 1 자료 참고
    
    #3)데이터 분류
    ## 설명변수 생성
    modelfit_X = hmeq.iloc[:,1:]
    modelfit_X.head()
    ## 반응변수 생성
    modelfit_y = hmeq.iloc[:,0]
    modelfit_y.head()
    ## 데이터 분류
    X_train, X_test, y_train, y_test = train_test_split(modelfit_X, modelfit_y, test_size=0.4, random_state=0)
    Xname = ['LOAN','MORTDUE','VALUE','YOJ','DEROG','DELINQ','CLAGE','NINQ','CLNO','DEBTINC']
    yname = ['good','bad']
    
    #4) Decision classifier  - default tree 생성
    cart0 = tree.DecisionTreeClassifier(criterion='gini',random_state=0) ## criterion = 'entropy'
    		###파라미터
    		### max_depth
    		### min_samles_split:노드를 split하기 위해 요구되는 최소한의 샘플 개수, 적을수록 복잡해짐
    		### min_impurity_decrease : 불순도를 감소시키도록 split, 작을수록 더 복잡해
    		### criterion : 계산하는 방법을 의미하며 그중 하나가 지니
    cart0.fit(X_train, y_train)
    dot_data = export_graphviz(cart0, out_file=None, feature_names=Xname,
        class_names=yname,filled=True, rounded=True, special_characters=True)
        ### filled: 그림에 색상을 넣을 것인가, rounded: 반올림을 진행할것인지, special_characters: 특수문자를 사용하는지
    graph = pydotplus.graph_from_dot_data(dot_data)
    Image(graph.create_png())
    
    #4) 정확도 비교
    y_prob0 = cart0.predict_proba(X_test)[:,1] ##예측값 출력 1번만 가져오란 의미
    
    #5) 앙상블 모델 - 배깅
    bagging = RandomForestClassifier(n_estimators=100, max_features=None, random_state=1234)
    bagging.fit(X_train, y_train)
    ##파라미터
    	###n_estimators:결정트리의 갯수를 지정
    	###min_samples_split:노드를 분할하기 위한 최소한의 "샘플 데이터수", 작게 설정할수록 과적합
    	###min_samples_leaf: 리프노드가 되기 위해 필요한 최소한의 샘플 데이터수
    	
    	###max_features: 최적의 분할을 위해 고려할 최대 feature 개수
    	### Default = 'auto' (결정트리에서는 default가 none이었음)
    	### int형으로 지정 →피처 갯수 / float형으로 지정 →비중
    	### sqrt 또는 auto : 전체 피처 중 √(피처개수) 만큼 선정
    	### log : 전체 피처 중 log2(전체 피처 개수) 만큼 선정
    	###max_depth : 트리의 최대 깊이, 완벽하게 클래스 값이 결정될 때 까지 분할또는 데이터 개수가 min_samples_split보다 작아질 때까지 분할
    
    #5) 정확도 측정
    bagging_pred = bagging.predict(X_test)
    accuracy = accuracy_score(y_test, bagging_pred) ##accuracy score 사용!!
    print(f'Mean accuracy score: {accuracy:.3}')
    ##Confusion Matrix
    cm = pd.DataFrame(confusion_matrix(y_test, bagging_pred), columns=yname, index=yname)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues") ##시각화
    ##Prediction probability
    y_prob_bag = bagging.predict_proba(X_test)[:,1] # 맞을 확률구함
    roc_auc_bag = roc_auc_score(y_test, y_prob_bag )
    
    #5-2) 앙상블 모델 - RF
    rf = RandomForestClassifier(n_estimators=100, max_features='sqrt', oob_score=True, random_state=1234)
    rf.fit(X_train, y_train)
    
    #5-2) 정확도 측정
    ## Accuracy
    rf_pred = rf.predict(X_test)
    accuracy = accuracy_score(y_test, rf_pred) # 정확도 
    print(f'Mean accuracy score: {accuracy:.3}')
    
    ## Confusion Matrix
    cm = pd.DataFrame(confusion_matrix(y_test, rf_pred), columns=yname, index=yname)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    
    y_prob_rf = rf.predict_proba(X_test)[:,1]
    roc_auc_rf = roc_auc_score(y_test, y_prob_rf )
    
    fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_prob_rf)
    
    #6) features 중요도 측정
    importances = rf1.feature_importances_
    
    for name, importance in zip(Xname, importances): print(name, "=", importance)
    indices = np.argsort(importances)
    plt.title('Feature Importances')
    plt.barh(range(len(indices)), importances[indices], color='b', align='center')
    plt.yticks(range(len(indices)),X_train.columns[indices])
    plt.xlabel('Relative Importance')
    plt.show()

## 스팸 분석 - 파이썬 & 스파크

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
nltk.download('stopwords') # 업그레이드 한번만 실행

#1)데이터 읽기 
df = pd.read_table("SMSSpamCollection",header=None,sep="\t")
df = df.rename(columns={0: 'label',1: 'messages'})

#2)stopWords제거

## stopWords 감지하는 함수
STOPWORDS = set(stopwords.words('english'))
def 불용어제거(텍스트):
    텍스트 = 텍스트.lower()
    텍스트 = re.sub(r'[^0-9a-zA-Z]', ' ', 텍스트)
    텍스트 = re.sub(r'\s+', ' ', 텍스트)
    텍스트 = " ".join(word for word in 텍스트.split() if word not in STOPWORDS)
    return 텍스트

##re모듈
df['processed'] = df['messages'].apply(불용어제거)


#3) 모델 트레이닝
## 모델 생성하는 함수 생성
X = df['processed']
y = df['label']
def SMS분류(model, X, y):
		#데이터 분류
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=316, shuffle=True, stratify=y) #stratify :데이터 층화 추출 => 각 계층에서 테스트 25% 트레인 75% 추출
    # model training
    pipeline_model = Pipeline([('vect', CountVectorizer()), # 우리가 가진 데이터에 함수를 연쇄 적용하여 최종적으로 "model"에 적용
                               ('tfidf',TfidfTransformer()),
                               ('clf', model)])
    pipeline_model.fit(x_train, y_train)# train값을 적용시켜서 모델을 생성함     
    print('Accuracy:', pipeline_model.score(x_test, y_test)*100)# test값을 가지고 모델의 score을 계산 (정확도)    
    
    y_pred = pipeline_model.predict(x_test)# 예측값 구함 
    print(classification_report(y_test, y_pred))
    
    cv_score = cross_val_score(pipeline_model, X, y, cv=5) #cv를 사용하여 score 구함 
    print("CV Score:", np.mean(cv_score)*100)

### Logistic regression
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
SMS분류(model, X, y)

### Naive Bayes
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
SMS분류(model, X, y)

### Support Vector Machine
from sklearn.svm import SVC
model = SVC(C=3)
SMS분류(model, X, y)

### Random Forest
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
SMS분류(model, X, y)

##예측값을 구하는 함수 
def SMS예측(model, x_train, y_train, x_test):
    pipeline_model = Pipeline([('vect', CountVectorizer()),
                               ('tfidf',TfidfTransformer()),
                               ('clf', model)])
    pipeline_model.fit(x_train, y_train)    
    y_pred = pipeline_model.predict(x_test)
    return y_pred

##데이터 분할
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=316, shuffle=True, stratify=y)
##모델 생성
model = SVC(C=3)
##예측값 함 
예측label = SMS예측(model, x_train, y_train, x_test)
print(type(y_test),type(예측label))
print(list(y_test[:10])) #리스트 형태로 변환하여 같이봄 (데이터 형태가 다르니까)
print(예측label[:10])

##정확도 출력
from sklearn.metrics import confusion_matrix
혼동행렬 = confusion_matrix(y_test, 예측label) # 테스트값과 예측값 비교하여 정확도 출력
print(혼동행렬)

from sklearn.metrics import precision_score, recall_score, f1_score
print("Precision:",precision_score(y_test, 예측label, pos_label="ham"))
print("Recall:",recall_score(y_test, 예측label, pos_label="ham"))
print("F1:",f1_score(y_test, 예측label, pos_label="ham"))

#1) countVector 사용
예제문장 =  ['This is the first document.', 'This document is the second document.', 
         'And this is the third one.', 'Is this the first document?']
## 토큰 생성 
vectorizer = CountVectorizer()
토큰개수 = vectorizer.fit_transform(예제문장)
vectorizer.get_feature_names_out() # 토큰들을 array로 출력

print(vectorizer.vocabulary_) # 각 토큰의 개수를 나타냄 0,1

## 단어의 가중치를 조정한 벡터를 생성
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2)) # 2개씩 데이터를 연달아 묶어서 표현-> 알파벳순으로 출력
토큰개수2 =  vectorizer2.fit_transform(예제문장)
vectorizer2.get_feature_names_out()

##
단어장 = ['this', 'document', 'first', 'is', 'second', 'the', 'and', 'one'] # 여기에 있는 단어들만 갖고 단어 빈도수를 계산하고 싶은 경우
vectorizer3 = CountVectorizer(vocabulary=단어장)
토큰개수3 =  vectorizer3.fit_transform(예제문장)
vectorizer3.get_feature_names_out() #토큰 출력 
print(토큰개수3.toarray()) # 토큰 빈도수 출력 (array형태로) 0,1 형태

## 불용어 제거
tfidf = TfidfVectorizer(stop_words = 'english').fit(예제문장)
print(tfidf.vocabulary_) # 각 토큰의 개수를 나타냄
print(tfidf.transform(예제문장).toarray()) # 가중치를 고려하여 해당 데이터의 비중을 표현 -> 가중치 값을 표현 
# -> 이 값을 비교하여 값이 거의 비슷한 데이터를 추출할 수 있음

##파이프라인 사용
파이프 = Pipeline([('count', CountVectorizer(vocabulary=단어장)), 
                 ('tfid', TfidfTransformer())]).fit(예제문장)
토큰개수  = 파이프['count'].transform(예제문장)
파이프['tfid'].idf_
파이프.transform(예제문장).toarray()

#############################################
#############################################
#PYSPARK#

#1) import문
from pyspark.ml.feature import CountVectorizer
countTokenizer = udf(lambda w: len(w), IntegerType())
##불용어
from pyspark.ml.feature import StopWordsRemover
##TF-IDF
from pyspark.ml.feature import IDF

#2) 데이터 읽기 및 변수명 변경
spark = SparkSession.builder.appName("nlp_nb").getOrCreate()
df = spark.read.csv("SMSSpamCollection",inferSchema=True,sep="\t")
df = df.withColumnRenamed("_c0","label").withColumnRenamed("_c1","messages")

df = df.withColumn("length",length(df["messages"]))
df.groupby("label").mean().show()

#3) 토큰화
from pyspark.ml.feature import StringIndexer
##
tokenizer = Tokenizer(inputCol="messages",outputCol="tokened")
stop_word_remover = StopWordsRemover(inputCol="tokened",outputCol="stoped")
count_vec = CountVectorizer(inputCol="stoped",outputCol="c_vec")
idf = IDF(inputCol="c_vec",outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol="label",outputCol="label_01")
##
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
cleaned = VectorAssembler(inputCols=['tf_idf','length'],outputCol="features")

##파이프라인
from pyspark.ml import Pipeline
파이프라인 = Pipeline(stages=[ham_spam_to_num, tokenizer,stop_word_remover,count_vec,idf,cleaned])
전처리기 = 파이프라인.fit(df)
전처리_df = 전처리기.transform(df)
전처리_df.show()

#3) 모델 생성 및 평가
최종_df = 전처리_df.select(['label_01','features']).withColumnRenamed("label_01","label")
(
train_df, test_df) = 최종_df.randomSplit([.75,.25], seed= 316)

### Naive Bayes Model
from pyspark.ml.classification import NaiveBayes, LogisticRegression, RandomForestClassifier, GBTClassifier, LinearSVC
nb = NaiveBayes()
적합모형 = nb.fit(train_df)
적합결과 = 적합모형.transform(test_df)

### MulticlassClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

eval = MulticlassClassificationEvaluator()
acc = eval.evaluate(적합결과)
print(f"Accuracy:{acc*100}")

## Linear & Logistic Regression -파이썬, 스파크
### 파이썬: Sklearn VS statsmodel

In [ ]:
#1) linear regression - 사이킷 런 VS Statsmodel!
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression  
import statsmodels.api as sm
##데이터 가져오기
whitedata = pd.read_csv("white.csv")
##반응 변수 설명변수 설정
y = whitedata.y  # whitedata["y"] 같은의미
x = whitedata.x1
plt.scatter(x,y,color="blue"); plt.show()# 산점도 그리기
##반응변수, 설명변수 설정2
x = whitedata["x1"].to_frame() #시리즈를 데이터프레임으로 변환
y = whitedata.y
## linearRegression 모델 생성
slm = LinearRegression().fit(x,y)       
w, b = slm.coef_[0], slm.intercept_ #coef = B1(추정된 가중치 벡터), intercept = B0 회귀계수 (추정된 상수항)
## 모델 시각화
plt.scatter(x,y,color="blue")
plt.plot([x0,x1],[w*x0+b,w*x1+b],'r'); plt.show()# 기울기 선 추가

#2) linear regression - statsmodels
## statsmodel의 사용
model = sm.OLS(y, sm.add_constant(x))   #OLS: 잔차제곱합을 최소화하는 가중치 벡터를 구하는 방법
results = model.fit()   # sm.add_constant(x) : 상수항 결합, 선형회귀모형에 반드시 필요
print(results.summary()) #OLS Regression 결과표를 출력

#2) 다중선형 회귀 모형 생성
## 반응변수 설명변수 생성
X = whitedata.drop(["y"],axis=1)
y = whitedata["y"]
##데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)
##regression 모델 생성
lrm = LinearRegression(n_jobs=-1)      # fit_intercept=True  # -1: 지금있는 모든 core를 모두 사용하라는 뜻
result = lrm.fit(X_train, y_train) 
print(result.coef_) #상관계수 출력 
##테스트 값으로 예측값 생성
forecast = lrm.predict(X_test) #새로운 입력 데이터에 대한 출력데이터 예측
##테스트값과 예측값 비교하여 정확도 분석 
accuracy = lrm.score(X_test, y_test)

## OLS regression -statsmodel 
model = sm.OLS(y_train, sm.add_constant(X_train)) 
results = model.fit()  
print(results.summary())# 결과표 출력 

#3) LASSO
## 모델 생성
from sklearn.linear_model import Lasso, Ridge
lasso = Lasso(alpha=0.1)
ridge = Ridge(alpha=0.1)
##모델 적합
lasso.fit(X_train, y_train)
print(lasso.intercept_ , lasso.coef_ )
##예측값 생성
forecast = lasso.predict(X_test) # 테스트 데이터로 예측함
print(forecast[0:10])

## training-validation procedure(반복 with train-test-split) # 무작위로 10번 test # 설명변수가 많은경우 모델을 fitting하면서 설명변수를 선택할 때 사용, 약간의 bias가 발생할 수 있다.
from sklearn.metrics import mean_squared_error
result = np.zeros((10,5))
a = [0.001, 0.002, 0.003, 0.004, 0.005] # "유의수준" 지정 -> 0.001을 선택하는 것이 좋음 
for step in range(10):
    X0, X1, y0, y1 = train_test_split(X, y, test_size=0.3)
    for choice in range(len(a)):
        lasso = Lasso(alpha=a[choice])      # sklearn.preprocessing.StandardScaler
        lasso.fit(X0, y0); forecast = lasso.predict(X1)
        result[step,choice] = mean_squared_error(y1,forecast)
print(result)
result = pd.DataFrame(result)
result.mean()

#4) Logistic Regression - statsmodel
##데이터 분류
whitedata["good"] = (whitedata.y > 5).astype(float) #good 칼럼의 데이터만 가져옴
y_binary = whitedata["good"] #target value
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.3, random_state=100)
##모델 생성
logit_sm = sm.Logit(y_train, sm.add_constant(X_train)).fit()
print(logit_sm.summary())

print("Parameters:",logit_sm.params)# sm모델의 파라미터 출력
print("Eta:", logit_sm.fittedvalues[0:10]
## 예측값 생성
muhat = logit_sm.predict(sm.add_constant(X_test)) #새로운 데이터로 타겟값을 예측 
print("prob:", muhat[0:10])
yhat = (muhat > 0.5).astype(int) # true =1, false=0 # y예측값을 구함
print(yhat[0:10])
##정확도 측정
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, yhat) #정확도 매트릭스를 사용하여 y의 테스트값과, 예측값인 yhat 값 비교
print ("Confusion Matrix : \n", cm) 
##TP FN   (실제 x 예측)
##FP TN
##accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, yhat))
      
##statsmodel - logit모델 생성
import statsmodels.formula.api as smf
model = "good ~ x2+x4+x6+x10+x11" #칼럼 X1~X11을 사용해여 모델 생성
logit_smf = smf.logit(formula=str(model),data=whitedata).fit()
logit_smf.summary() ##logistic regression의 결과를 표로 출력

##sklearn linear model 생성
from sklearn.linear_model import LogisticRegression
logitS = LogisticRegression(penalty=None)  ## C=1/alpha=1, penality="l2" {"l1","l2","elasticnet",None}
logitS.fit(X_train, y_train)
print(logitS.coef_, logitS.intercept_)
logitL = LogisticRegression(penalty="l1",solver="saga")  # LASSO , C = 1/alpha, solver를 saga로 적용 후 사용가능함, 라쏘는 기준값 이하로 내려가면 0으로 보내버림
logitL.fit(X_train, y_train)
##정확도 예측
muL = logitL.predict_proba(X_test) # 뮤 햇을 구하여 muL 생성 (예측 확률 구함)
yhat = logitL.predict(X_test)  # 예측값을 생성 
print(muL[0:10])
print(yhat[0:10]) #결과값은 0 ,1 로 표현
##예측값과 정답을 비교하여 정확도 출력 
logitL.score(X_test, y_test)
      
##########################################
##########################################
##SPARK##
##regression


#1) 스파크 생성
from pyspark.sql import SparkSession

spark =  SparkSession.builder.appName("wine").getOrCreate()
white  = spark.read.csv("white.csv",inferSchema=True,header=True)

#2) 컬럼명으로 구성된 list 생성 
from pyspark.ml.feature import VectorAssembler
설명변수 = list(white.columns)# 칼럼명만 들어있는 리스트 

변수묶음 = VectorAssembler(inputCols=설명변수,outputCol="features")
변환자료  = 변수묶음.transform(white) #각 칼럼의 값이 list형태로 행기준으로 묶임

#3)Linear Regression
##회귀자료 생성
회귀자료 = 변환자료.select(["features","y"])#설명변수, 반응변수
##자료 분할
from pyspark.ml.regression import LinearRegression
train_data, test_data = 회귀자료.randomSplit([0.7, 0.3], 316)
##모델 생성
회귀분석  = LinearRegression(featuresCol="features",labelCol="y").fit(train_data)
회귀분석.coefficients # 상관계수
회귀분석.intercept #회귀계수 (상수함)
pred = 회귀분석.evaluate(test_data) 
pred.predictions.show() #테스트자료로 예측값(반응변수) 출력 
# 결정계수, MSE(모분산 추정)
print(pred.r2, pred.meanSquaredError)

######### PYSPARK logistic 모델 생성


##문자변수만 모아 생성
from pyspark.sql.types import StringType
문자변수 = [변수.name for 변수 in churn_df.schema.fields if isinstance(변수.dataType, StringType)] #데이터프레임의 범주의 타입이 문자인지 확인하고 맞다면 출력
문자변수 # 앞으로 이 문자변수를 갖고 숫자 변수로 변화해주는 작업을 할것임

#1) 문자변수를 숫자값을 가진 변수로 변환
from pyspark.ml.feature import StringIndexer, OneHotEncoder
##-1 StringIndexer
indexer  = StringIndexer(inputCols=문자변수, #내가 바꿀 변수(1개 이상의 값을 가진 경우)
                         outputCols=["{}_SI".format(c) for c in 문자변수]) # 컬럼명을 변경
encode_df  = indexer.fit(churn_df).transform(churn_df) #기존 데이터프레임에 새로운 변수값들이 추가 -> 타입이 숫자로 변환
encode_df.printSchema()

##-2 OneHotEncoder : 숫자 변경
oneHot = OneHotEncoder(inputCols = ["{}_SI".format(c) for c in 문자변수],
                        outputCols =["{}_encoder".format(c) for c in 문자변수], dropLast =False )
oneHot_df = oneHot.fit(encode_df).transform(encode_df)
oneHot_df.limit(5).show()
##숫자로 구성된 칼럼 생성
설명변수 = ["SeniorCitizen","MonthlyCharges"]+["{}_SI".format(c) for c in 문자변수] #기존의 숫자칼럼 + 내가 변환한 숫자칼럼 => 하나의 데이터프레임으로 형성
설명변수 = 설명변수[0:-1]

#2) features 생성 -> 각 칼럼의 값들을 행 기준으로 리스트로 묶
from pyspark.ml.feature import VectorAssembler

변수묶음 = VectorAssembler(inputCols=설명변수,outputCol="features") #output은 하나의 묶여진 형태로 나옴
변환자료  = 변수묶음.transform(encode_df)
변환자료.select("features","Churn_SI").show()
분류자료 = 변환자료.select(["features","Churn_SI"])


#3)logistic regression model 생성
from pyspark.ml.classification import LogisticRegression
##데이터 분류
train_data, test_data =분류자료.randomSplit([0.7, 0.3], 316)
## 모델 생성 
분석모형 =  LogisticRegression(labelCol="Churn_SI").fit(train_data) # 반응변수:labelCol
분석모형.summary
##반응변수의 예측값 출력
분석모형.summary.predictions.show(truncate=False)
##기술 통계값 계산
분석모형.summary.predictions.describe().show()

#4) 모델 평가
from pyspark.ml.evaluation import BinaryClassificationEvaluator
##테스트 데이터로 평
예측 = 분석모형.evaluate(test_data) # 테스트 데이터로 모델을 평가해야함
예측.predictions.show()
##함수를 사용하여 정확도 평
평가 = BinaryClassificationEvaluator(rawPredictionCol="prediction",labelCol="Churn_SI") ## labelCol = 타겟값
auc = 평가.evaluate(예측.predictions) # 예측값 출력
auc

## Classification -sklearn
### 자세한 방식보다 전체적으로 모델 어떻게 생성하는지 설명

In [ ]:
#1) 데이터 가져오기
from sklearn.datasets import load_iris
from sklearn import tree
iris = load_iris()

#2) 모델 생성
clf = tree.DecisionTreeClassifier().fit(iris.data, iris.target) #(설명변수, 반응변수)

###Graphviz 설치

#3)decision tree 사진을 생성
import pydotplus
from IPython.display import Image
dot_data = tree.export_graphviz(clf, out_file=None,
                                 feature_names=iris.feature_names,
                                 class_names=iris.target_names,
                                filled=True, rounded=True,
                                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf("iris.pdf") ##pdf로 저장
Image(graph.create_png())

#4) 앙상블. 모형 소개 -> 배깅, 부스팅, 랜덤포레스트
##모델. 생성을 위한 임포트
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import graphviz#from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier ##sklearn에서 사용!
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.datasets import load_iris

## 데이터 분할 from sklearn.model_selection import train_test_split
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=100)

#4-1) Baggin Classifier 생성
m_bagging = BaggingClassifier()
m_bagging.fit(X_train,y_train)

##테스트값 예측
pred_bagging = m_bagging.predict(X_test)
## 실제 y값과 테스트값 비교(confusion matrix 사용)
cf_bagging=confusion_matrix(y_test,pred_bagging)
print('Bagging Confusion Matrix')
print(cf_bagging)

## 실제 y값과 테스트값 비교-> 정확도 출력(score)
score_bagging=m_bagging.score(X_test, y_test)
print('Bagging Accuracy')
print(score_bagging)

#4-2)boosting
m_boosting = GradientBoostingClassifier()
m_boosting.fit(X_train,y_train)
## 테스트 값으로 label예측
pred_boosting = m_boosting.predict(X_test)
## confusion matrix로 정확도 예측
cf_boosting = confusion_matrix(y_test,pred_boosting)
print('Boosting Confusion Matrix')
print(cf_boosting)
## 테스트 값과 실제 y값을 비교하여 score 생성
score_boosting = m_boosting.score(X_test, y_test)
print('Boosting Accuracy'
)print(score_boosting)

#4-3) RF
m_RF = RandomForestClassifier(n_estimators=1000) ##1000번 반복함
m_RF.fit(X_train,y_train)
##테스트 값으로 label 예측
pred_RF = m_RF.predict(X_test)
##confusion matrix 사용하여 테스트값과 y값 비교
cf_RF = confusion_matrix(y_test,pred_RF)
print('Random Forest Confusion Matrix')
print(cf_RF)
## 정확도 출력
score_RF = m_RF.score(X_test, y_test)
print('Random Forest Accuracy')print(score_RF)
## 모델링에서 많이 언급된 변수를 추출
###세번째 변수가 가장 많이 언급
importances = m_RF.feature_importances_
importances
##변동성 측정 
std = np.std([m_RF.feature_importances_ for tree in m_RF.estimators_], axis=0)
변수명 = [f"feature {i}" for i in range(iris.data.shape[1])]
변수명
## 사용된 변수 importance 시각화 
forest_importances = pd.Series(importances, index=변수명)
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()


## Clustering - kmeans Cluster

In [ ]:
from sklearn.preprocessing import scale
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

iris = load_iris() ## array형태로 저장되어있음

iris_df = pd.DataFrame(data=iris.data, columns=["sepal_length","sepal_width","petal_length","petal_width"]) # 컬럼 이름 지정
iris_df.head()

kmeans = KMeans(n_clusters=3, max_iter=300, random_state=316) #3개의 클러스털 생성, 316번동안 모델링함
kmeans.fit(iris_df)# 모델생성 후 fit 시킴

print(kmeans.labels_) # 예측한값  라벨 확인

iris_df["target"] = iris.target # 데이터에 저장되어있던 정답값
iris_df["cluster"] = kmeans.labels_
iris_result = iris_df.groupby(["target","cluster"])["sepal_length"].count() #target: 정답값, cluster:예측값 비교
print(iris_result)

## 군집화 평가 - 실루엣 계수 사용
from sklearn.metrics import silhouette_samples, silhouette_score

iris_df["silhouette"] = silhouette_samples(iris.data, iris_df["cluster"])

ave_score = silhouette_score(iris.data,iris_df["cluster"])
print("SAS:{0:.4f}".format(ave_score))
iris_df.head()

iris_df.groupby("cluster")["silhouette"].mean() # 그룹0은 잘 분류되었지만. 1과 2는 다소 잘 분류
 
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=3, random_state=316).fit(iris.data)
iris_df["gmm"] = gmm.predict(iris.data)
iris_df.head()

iris_result = iris_df.groupby(["target"])["gmm"].value_counts()
print(iris_result) ## target: 정답, gmm: 예측값 (그룹 명이 변경될 수도 있음) , 여기 아래 예시에선 1과 0클러스터를 이름을 바꿔서 분류했다고 볼 수 있음 -> 정확도 높음 



## classification - SVM

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

#1) 데이터 불러오기
from sklearn import datasets
iris = datasets.load_iris()
## 데이터 추출
x = iris.data[:, [0, 2]] 
y = iris.target
##학습자료와 검증자료 분리
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.3, random_state=1, stratify=y)
##데이터 표준화
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
sc.fit(x_train)
###numpy.ndarray 형식으로 출력
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)
###x_train_std의평균과 표준편차 - array 형태
print('Mean of x_train_std:',np.mean(x_train_std[:,0]), np.mean(x_train_std[:,1]))
print('Stdof x_train_std:',np.std(x_train_std[:,0]), np.std(x_train_std[:,1]))

#2) SVM모델 생성
## C가작은경우(오분류를관대하게허용)

## 파라미터: SVC 클래스에 kernel='linear', 시드넘버는1, C=0.1로 설정
### C값이 달라지면 정확도가 달라짐, C가 커졌을 때 정확도 높아짐
### gamma =1, (작은 값) = 100 : 오버피팅 발생 !!!
from sklearn.svm import SVC
svm_smallc= SVC(kernel='linear', random_state=1, C=0.1) ## SVC: classification ,SVR: regression
svm_smallc.fit(x_train_std, y_train)
##표준화된 것과 안된 것
x_combined_std= np.vstack((x_train_std, x_test_std))
y_combined= np.hstack((y_train, y_test))

#3) 데이터와 결정 경계그림 그리기
from mlxtend.plotting import plot_decision_regions

plot_decision_regions(x_combined_std, y_combined, clf=svm_smallc)
plt.xlabel('petal length [standardized]') #x축
plt.ylabel('petal width [standardized]')  #y축
plt.legend(loc='upper left')

#4) 예측값과 관측값 비교
y_pred= svm_smallc.predict(x_test_std)
y_pred,(y_test!= y_pred)
print('Misclassified samples: %d' % (y_test!= y_pred).sum()) #10개의 샘플이 잘못 분류되었다고 출력

#5) 정확도 계산
from sklearn.metrics import accuracy_score
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred)) # 정답과 추정값 비교
